In [1]:
import os
import re
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import mapping
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from scipy import ndimage
from functools import partial

In [2]:

# =========================================================
# 2) TRÍCH DATETIME TỪ FILENAME
# =========================================================
def extract_datetime_from_filename(path):
    filename = os.path.basename(path)

    # Kiểu 1: CAPE_20190401000000.tif
    m14 = re.search(r"(\d{14})", filename)
    if m14:
        return pd.to_datetime(m14.group(1), format="%Y%m%d%H%M%S", errors="coerce")

    # Kiểu 2: B04B_20190401.Z0000_TB.tif
    m_date = re.search(r"(\d{8})", filename)
    m_z = re.search(r"Z(\d{4})", filename)

    if m_date:
        date = m_date.group(1)
        if m_z:
            return pd.to_datetime(date + m_z.group(1), format="%Y%m%d%H%M", errors="coerce")
        return pd.to_datetime(date, format="%Y%m%d", errors="coerce")

    return pd.NaT


# =========================================================
# 3) LIST FILE
# =========================================================
def list_all_files(root):
    out = []
    for dp, _, files in os.walk(root):
        for f in files:
            if f.endswith(".tif") or f.endswith(".TIF"):
                out.append(os.path.join(dp, f))
    return out

In [3]:
# =========================================================
# 1) LOAD SHAPEFILE HÀ TĨNH
# =========================================================
shp_path = "gadm41_VNM_shp"
vnm_gdf = gpd.read_file(shp_path)

# Lọc Hà Tĩnh
ht_gdf = vnm_gdf[vnm_gdf['VARNAME_1'] == 'Ha Tinh']

ht_union = ht_gdf.geometry.union_all()
ht_crs = ht_gdf.crs

In [4]:
# =========================================================
# 4) NHANH HƠN: FILL NODATA = NEAREST NEIGHBOR (KHÔNG DÙNG griddata)
# =========================================================
def fast_fill_nodata_nearest(arr):
    mask = np.isnan(arr) | np.isinf(arr)
    if not mask.any():
        return arr

    # Distance transform để tìm pixel valid gần nhất
    dist, (inds_y, inds_x) = ndimage.distance_transform_edt(mask,
        return_indices=True
    )
    return arr[inds_y, inds_x]

In [13]:
def extract_HaTinh_bbox_all_pixels(path, root):

    try:
        with rasterio.open(path) as src:
            data = src.read(1).astype(float)
            nodata = src.nodata
            transform = src.transform
            src_crs = src.crs

        # --- chuẩn hóa NODATA ---
        data[data == nodata] = np.nan
        data[data == -9999] = np.nan
        data[np.isinf(data)] = np.nan

        # --- fill nodata bằng nearest ---
        data = fast_fill_nodata_nearest(data)

        # --- reproject Hà Tĩnh sang CRS raster ---
        if src_crs != ht_crs:
            geom = ht_gdf.to_crs(src_crs).geometry.union_all()
        else:
            geom = ht_union

        # --- BƯỚC 1: Lấy Bounding Box "Thô" (Loose BBox) ---
        # Cái này sẽ ra 23x36 (hơi dư)
        bbox = geom.bounds
        min_row_loose, min_col_loose = rasterio.transform.rowcol(transform, bbox[0], bbox[3])
        max_row_loose, max_col_loose = rasterio.transform.rowcol(transform, bbox[2], bbox[1])

        # Clamp vào phạm vi ảnh
        min_row_loose = int(max(0, min_row_loose))
        min_col_loose = int(max(0, min_col_loose))
        max_row_loose = int(min(data.shape[0] - 1, max_row_loose))
        max_col_loose = int(min(data.shape[1] - 1, max_col_loose))

        # --- BƯỚC 2: Tạo Mask cho vùng "Thô" này ---
        # Tính kích thước vùng thô
        height_loose = max_row_loose - min_row_loose + 1
        width_loose = max_col_loose - min_col_loose + 1

        # Tạo transform cho riêng vùng thô này để làm mask
        window_transform = rasterio.windows.transform(
            rasterio.windows.Window(min_col_loose, min_row_loose, width_loose, height_loose),
            transform
        )

        # Tạo mask: True nếu là Hà Tĩnh, False nếu là ngoài
        mask_subset = geometry_mask(
            [mapping(geom)],
            invert=True, # True = Inside
            out_shape=(height_loose, width_loose),
            transform=window_transform
        )

        # --- BƯỚC 3: Tìm Bounding Box "Tinh" (Tight BBox) từ Mask ---
        # Tìm các chỉ số hàng/cột nơi mask là True
        valid_rows, valid_cols = np.where(mask_subset)

        if len(valid_rows) > 0:
            # Tìm biên của các pixel thực sự thuộc Hà Tĩnh
            trim_min_r, trim_max_r = valid_rows.min(), valid_rows.max()
            trim_min_c, trim_max_c = valid_cols.min(), valid_cols.max()

            # Tính lại chỉ số row/col trên ảnh gốc (cộng offset)
            min_row = min_row_loose + trim_min_r
            max_row = min_row_loose + trim_max_r
            min_col = min_col_loose + trim_min_c
            max_col = min_col_loose + trim_max_c
        else:
            # Trường hợp không có pixel nào (hiếm), giữ nguyên loose
            min_row, max_row = min_row_loose, max_row_loose
            min_col, max_col = min_col_loose, max_col_loose
            print(f"WARNING: No pixel center inside geometry: {path}")

        # --- BƯỚC 4: Tạo grid và lấy dữ liệu theo BBox mới (21x34) ---
        rows = np.arange(min_row, max_row + 1)
        cols = np.arange(min_col, max_col + 1)

        row_grid, col_grid = np.meshgrid(rows, cols, indexing='ij')
        rows_flat = row_grid.flatten()
        cols_flat = col_grid.flatten()

        vals = data[rows_flat, cols_flat]

        # Yêu cầu của bạn: Tất cả pixel trong Bounding Box (mới) đều là True
        in_ha_tinh = np.full_like(vals, True, dtype=bool)

        # --- Các bước còn lại như cũ ---
        lons, lats = rasterio.transform.xy(transform, rows_flat, cols_flat, offset='center')
        ts = extract_datetime_from_filename(path)
        rel = os.path.relpath(path, root)
        var = rel.split(os.sep)[0]

        return pd.DataFrame({
            "variable": var,
            "timestamp": ts,
            "row": rows_flat,
            "col": cols_flat,
            "lon": lons,
            "lat": lats,
            "value": vals,
            "in_ha_tinh": in_ha_tinh
        })

    except Exception as e:
        print("ERROR:", path, e)
        return pd.DataFrame() # Trả về rỗng để dễ xử lý

In [16]:

# =========================================================
# 6) MAIN
# =========================================================
def tif2csv():

    # out_csv = "csv_data/RADAR_hatinh.csv"
    # root = "DATA_SV/Precipitation/Radar"

    # out_csv = "csv_data/HIMA_hatinh_rec.csv"
    # root = "DATA_SV/Hima"

    out_csv = "csv_data/ERA5_hatinh_rec.csv"
    root = "DATA_SV/ERA5"

    os.makedirs(os.path.dirname(out_csv), exist_ok=True)

    files = list_all_files(root)
    print("Tổng file tìm thấy:", len(files))

    if os.path.exists(out_csv):
        os.remove(out_csv)

    func = partial(extract_HaTinh_bbox_all_pixels, root=root)
    results = []

    with ThreadPoolExecutor(max_workers=10) as pool:
        futures = [pool.submit(func, f) for f in files]

        for f in tqdm(as_completed(futures), total=len(futures), desc="Process"):
            try:
                df = f.result()
                if df is not None and not df.empty:
                    results.append(df)
            except Exception as e:
                print("Thread error:", e)

    if results:
        final = pd.concat(results, ignore_index=True)
        final.to_csv(out_csv, index=False)
        print("DONE! Tổng pixel =", len(final))
    else:
        print("Không có data.")

tif2csv()

Tổng file tìm thấy: 58560


Process: 100%|██████████| 58560/58560 [19:09<00:00, 50.94it/s] 


DONE! Tổng pixel = 41811840


In [15]:
df_all1 = pd.read_csv('csv_data/HIMA_hatinh_rec.csv')

min_row1, max_row1 = df_all1["row"].min(), df_all1["row"].max()
min_col1, max_col1 = df_all1["col"].min(), df_all1["col"].max()
n_row1 = int(max_row1 - min_row1 + 1)
n_col1 = int(max_col1 - min_col1 + 1)

print(f"min_row: {min_row1}")
print("min_col: ", min_col1)
print("max_row: ", max_row1)
print("max_col: ", max_col1)
print("n_row: ",n_row1)
print("n_col: ",n_col1)


min_row: 59
min_col:  103
max_row:  79
max_col:  136
n_row:  21
n_col:  34


In [12]:
n_row1 = max_row1 - min_row1 + 1
n_col1 = max_col1 - min_col1 + 1
print("n_row: ",n_row1)
print("n_col: ",n_col1)

n_row:  23
n_col:  36


In [17]:
df_all = pd.read_csv('csv_data/ERA5_hatinh_rec.csv')

min_row, max_row = df_all["row"].min(), df_all["row"].max()
min_col, max_col = df_all["col"].min(), df_all["col"].max()
n_row = int(max_row - min_row + 1)
n_col = int(max_col - min_col + 1)

print(f"min_row: {min_row}")
print("min_col: ", min_col)
print("max_row: ", max_row)
print("max_col: ", max_col)
print("n_row: ",n_row)
print("n_col: ",n_col)

min_row: 59
min_col:  103
max_row:  79
max_col:  136
n_row:  21
n_col:  34


In [9]:
n_row = max_row - min_row + 1
n_col = max_col - min_col + 1
print("n_row: ",n_row)
print("n_col: ",n_col)

n_row:  23
n_col:  36


In [18]:
def create_x_y_selected_features(list_path, selected_features=None):
    """
    Phiên bản tối ưu: Sửa lỗi index float, cố định thứ tự band và tăng tốc độ xử lý.
    """
    if selected_features is None:
        selected_features = ['B04B', 'B10B', 'B11B', 'B16B', 'IRB',
                             'CAPE', 'R850', 'TCWV', 'U850', 'I2B', 'TCLW', 'TCW']

    # 1) Đọc CSV và gộp
    dfs = []
    print("[B1] Đọc CSV...")
    for p in tqdm(list_path, desc="Đọc file CSV"):
        # Mẹo: Xác định dtype ngay lúc đọc để tiết kiệm bộ nhớ nếu file lớn
        df = pd.read_csv(p)
        df["variable"] = df["variable"].astype(str)
        # Gán nhãn y
        df.loc[df["variable"].isin(['2019', '2020']), "variable"] = 'y'
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    # 2) Min/Max row/col → tạo hình chữ nhật
    min_row, max_row = df_all["row"].min(), df_all["row"].max()
    min_col, max_col = df_all["col"].min(), df_all["col"].max()
    n_row = int(max_row - min_row + 1)
    n_col = int(max_col - min_col + 1)

    # 3) Xác định danh sách band cần thiết (QUAN TRỌNG: Phải Sort để cố định thứ tự)
    required_bands_list = sorted(list(set(selected_features + ['y'])))
    required_bands_set = set(required_bands_list)

    # 4) Lọc Timestamp hợp lệ (TỐI ƯU HÓA TỐC ĐỘ)
    # Chỉ giữ lại các dòng thuộc các variable quan tâm để đếm cho nhanh
    df_check = df_all[df_all["variable"].isin(required_bands_set)]

    # Đếm số lượng variable unique trong mỗi timestamp
    # Nếu timestamp T1 có đủ 13 variable -> count sẽ là 13
    ts_counts = df_check.groupby("timestamp")["variable"].nunique()

    # Lấy ra các timestamp có số lượng variable bằng đúng số lượng yêu cầu
    valid_ts_index = ts_counts[ts_counts == len(required_bands_set)].index
    ts_valid = sorted(list(valid_ts_index))

    # 5) Chuẩn bị dữ liệu để đổ vào Tensor
    # Tạo mapping index (Dictionary comprehension)
    t_to_idx = {t: i for i, t in enumerate(ts_valid)}
    b_to_idx = {b: i for i, b in enumerate(required_bands_list)} # Dùng list đã sort

    # Lọc dữ liệu chính thức:
    # - Chỉ lấy timestamp hợp lệ
    # - Chỉ lấy variable nằm trong required_bands (Bước này sửa lỗi Index Float)
    df_valid = df_all[
        (df_all["timestamp"].isin(ts_valid)) &
        (df_all["variable"].isin(required_bands_set))
    ].copy()

    # Map sang index (Ép kiểu int rõ ràng để tránh lỗi)
    df_valid["t_idx"] = df_valid["timestamp"].map(t_to_idx).astype(int)
    df_valid["b_idx"] = df_valid["variable"].map(b_to_idx).astype(int)
    df_valid["r_idx"] = (df_valid["row"] - min_row).astype(int)
    df_valid["c_idx"] = (df_valid["col"] - min_col).astype(int)

    # 6) Đổ dữ liệu vào Tensor (Vectorized - Không cần vòng lặp)
    print("[B4] Đổ dữ liệu vào Tensor...")
    tensor = np.zeros((len(ts_valid), len(required_bands_list), n_row, n_col), dtype=float)

    # Numpy Advanced Indexing: Nhanh hơn loop rất nhiều
    tensor[df_valid["t_idx"].values,
           df_valid["b_idx"].values,
           df_valid["r_idx"].values,
           df_valid["c_idx"].values] = df_valid["value"].values

    # 7) Tách X và y
    y_idx = b_to_idx['y']
    # Lấy mảng X indices: loại bỏ index của y
    x_indices = [i for i, b in enumerate(required_bands_list) if b != 'y']

    y = tensor[:, [y_idx], :, :]
    x = tensor[:, x_indices, :, :]

    return x, y, ts_valid, required_bands_list, (min_row, max_row), (min_col, max_col)

In [19]:
def luuTensor():
    list_file = [
        'csv_data/HIMA_hatinh_rec.csv',
        'csv_data/ERA5_hatinh_rec.csv',
        'csv_data/RADAR_hatinh.csv'
    ]
    x, y, timestamps, x_bands, row_range, col_range = create_x_y_selected_features(list_file)


    np.save("csv_data/x_hatinh_rec.npy", x)

luuTensor()

[B1] Đọc CSV...


Đọc file CSV: 100%|██████████| 3/3 [02:23<00:00, 47.84s/it]


[B4] Đổ dữ liệu vào Tensor...


In [22]:
X = np.load('csv_data/x_hatinh_rec.npy')

In [23]:
print(X.shape)

(1223, 12, 21, 34)
